For those cases where the radiance maps exist as npy arrays, but the code broke before making the full report showing all the data, at least the resulting npy arrays of the radiance maps can be outputted easily using this script

Outputs the radiance maps into a pdf report
No internal python modules needed; all internal code self-contained.

In [5]:
import os
import numpy as np
import matplotlib.pyplot as plt
from reportlab.lib.pagesizes import letter
from reportlab.platypus import SimpleDocTemplate, Image, Paragraph, PageBreak
from reportlab.lib.styles import getSampleStyleSheet
from reportlab.lib.units import inch
import io
from datetime import datetime

def plot_radiance_maps(radiance_map, title):
    fig, (ax1, ax2, ax3) = plt.subplots(1, 3, figsize=(15, 5))
    
    # Linear scale radiance map
    im1 = ax1.imshow(radiance_map, cmap='viridis')
    ax1.set_title(f'{title}\nLinear Scale')
    plt.colorbar(im1, ax=ax1)
    
    # Log scale radiance map
    with np.errstate(divide='ignore', invalid='ignore'):
        log_radiance = np.log10(radiance_map)
    im2 = ax2.imshow(log_radiance, cmap='viridis')
    ax2.set_title(f'{title}\nLog Scale')
    plt.colorbar(im2, ax=ax2)
    
    # Histogram of radiance values
    ax3.hist(radiance_map.ravel(), bins=100, range=(np.percentile(radiance_map, 1), np.percentile(radiance_map, 99)))
    ax3.set_title(f'{title}\nHistogram')
    ax3.set_xlabel('Radiance')
    ax3.set_ylabel('Frequency')
    
    plt.tight_layout()
    
    # Save the plot to a BytesIO object
    buf = io.BytesIO()
    plt.savefig(buf, format='png', dpi=300)
    buf.seek(0)
    plt.close()
    
    return buf

def generate_radiance_report(data_folder, output_file):
    # Get all radiance map files
    radiance_files = [f for f in os.listdir(data_folder) if f.endswith('_radiance_map.npy')]
    
    # Create a PDF document
    doc = SimpleDocTemplate(output_file, pagesize=letter, rightMargin=72, leftMargin=72, topMargin=72, bottomMargin=18)
    
    story = []
    styles = getSampleStyleSheet()
    
    for radiance_file in radiance_files:
        # Load radiance map
        radiance_map = np.load(os.path.join(data_folder, radiance_file))
        
        # Create plots
        plot_buffer = plot_radiance_maps(radiance_map, radiance_file[:-4])  # Remove '.npy' from title
        
        # Add title and plot to the PDF
        story.append(Paragraph(radiance_file, styles['Heading1']))
        story.append(Image(plot_buffer, width=7*inch, height=3*inch))
        story.append(PageBreak())
    
    # Build the PDF
    doc.build(story)
    print(f"Report saved as {output_file}")

if __name__ == "__main__":
    # Specify the directory containing the radiance map .npy files
    data_folder = "data/240329_Water_immersed"
    experiment_title = 'Water_immersed'

    
    # Specify the output PDF file
    timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
    output_file = f'{data_folder}/radiance_maps_report_{experiment_title}_{timestamp}.pdf'

    generate_radiance_report(data_folder, output_file)

Report saved as data/240329_Water_immersed/radiance_maps_report_Water_immersed_20240728_172610].pdf
